In [4]:
import torch
from torch import nn, optim
from torchvision import datasets, transforms
import torch.nn.functional as F
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
])
train_data = datasets.CIFAR10(
    root="data",
    train=True,
    download=True,
    transform=transform
)
test_data = datasets.CIFAR10(
    root="data",
    train=False,
    download=True,
    transform=transform
)

batch_size = 64
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3,16,3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(16,32,3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )
        self.fc = nn.Linear(32*8*8, 10)
    def forward(self, x):
        x = self.net(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

model = Model().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
epochs = 10

for epoch in range(epochs):
    print(f'Epoch: {epoch}')
    for X, Yd in train_loader:
        optimizer.zero_grad()
        X = X.to(device)
        Yd = Yd.to(device)
        Y_logit = model(X)
        loss = F.cross_entropy(Y_logit, Yd)
        loss.backward()
        optimizer.step()
    with torch.no_grad():
        loss_all = 0
        for X, Yd in test_loader:
            X = X.to(device)
            Yd = Yd.to(device)
            Y_logit = model(X)
            loss = F.cross_entropy(Y_logit, Yd)
            loss_all += loss
        print(loss_all / len(test_loader))


Files already downloaded and verified
Files already downloaded and verified
cuda
Epoch: 0
tensor(1.2341, device='cuda:0')
Epoch: 1
tensor(1.0553, device='cuda:0')
Epoch: 2
tensor(0.9976, device='cuda:0')
Epoch: 3
tensor(0.9510, device='cuda:0')
Epoch: 4
tensor(0.9641, device='cuda:0')
Epoch: 5
tensor(0.9608, device='cuda:0')
Epoch: 6
tensor(0.9043, device='cuda:0')
Epoch: 7
tensor(0.9083, device='cuda:0')
Epoch: 8
tensor(0.9123, device='cuda:0')
Epoch: 9
tensor(0.9160, device='cuda:0')
